# Installation

In [2]:
HOME_FOLDER = '/kaggle/working'
TEMP_FOLDER = '/kaggle/temp'
update = False

checkpoints = f'{HOME_FOLDER}/Fooocus/models/checkpoints'
temp_models = f'{HOME_FOLDER}/Fooocus/models/checkpoints/temp_models'

!mkdir $TEMP_FOLDER

import os
from os import path

%cd $HOME_FOLDER

if not path.exists('DeFooocus'):
    get_ipython().system('git clone https://github.com/ehristoforu/DeFooocus.git')
else:
    get_ipython().system(f'find {HOME_FOLDER}/DeFooocus/models/checkpoints -maxdepth 1 -type l -delete') # delete any symlinks to temp models from a previous run
    
%cd DeFooocus

if update:
    get_ipython().system('git pull')
    
!pip install -r requirements_versions.txt 
!pip install torch torchvision --force-reinstall --index-url https://download.pytorch.org/whl/cu117
!rm -rf /opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/PyYAML-6.0.1.dist-info
!pip install pyyaml pytz 

!mamba install openssh -y

mkdir: cannot create directory ‘/kaggle/temp’: File exists
/kaggle/working
Cloning into 'DeFooocus'...
remote: Enumerating objects: 6129, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 6129 (delta 87), reused 105 (delta 58), pack-reused 5983
Receiving objects: 100% (6129/6129), 45.94 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (3337/3337), done.
/kaggle/working/DeFooocus
  Obtaining dependency information for torchsde==0.2.5 from https://files.pythonhosted.org/packages/73/8d/efd3e7b31ea854d0bd6886aa3cf44914adce113a6d460850af41ac1dd4dd/torchsde-0.2.5-py3-none-any.whl.metadata
  Obtaining dependency information for einops==0.4.1 from https://files.pythonhosted.org/packages/df/bf/18c6c753fe699c44e3d6013d75074e15f139f9a9adbaeb108f61d73d18d1/einops-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers==4.30.2 from https://files.pythonhosted.org/packages/5b/0b/e45d26ccd28568013523e04f32543

# Start the WebUI

**Option 1: Starting the Web UI with ngrok**  
* Make sure to put your ngrok token in the Ngrok_token variable. The token can be obtained from https://ngrok.com
* If you have a static domain, put your ngrok domain in the Ngrok_domain variable.
* Wait for the line that says "App started successful. Use the app with http://127.0.0.1:7865/ or 127.0.0.1:7865" 
    * If the port is no longer 7865 in the latest version, please change the port variable as well
* Visit your ngrok URL (either your static domain, or the ngrok url displayed in the output)

In [ ]:
# Starting the Web UI with ngrok

# --- Variables ---

Ngrok_token = "2e7d3dRfazYuxRiwLEMdHXzwYFv_5UYoY9T2Jh3Gno3xTCo2h" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)
# Example: Ngrok_token = "2Fw13n4GcJT12g7mSDUC62cdNGb_5svdjf3Gg5vfhr4nGr5gF"

Ngrok_domain = "" # optional, leave empty if you don't have a domain

port = 7865

# -----------------


!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys
import gc

gc.collect()

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(port , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system(f"python {HOME_FOLDER}/DeFooocus/entry_with_update.py ")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

  Obtaining dependency information for pyngrok from https://files.pythonhosted.org/packages/cb/55/68b89d526e8331724665dcded0a32a76d73d6bcac41cc56084fda8e25486/pyngrok-7.1.6-py3-none-any.whl.metadata
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
https://90e7-34-141-155-127.ngrok-free.app                                                          
Already up-to-date
Update succeeded.
[System ARGV] ['/kaggle/working/DeFooocus/entry_with_update.py']
Python 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
Fooocus version: 0.1
Version mismatch for Pillow: Installed version 10.2.0 does not meet requirement Pillow==9.2.0
Instal

***   
**Option 2: Starting the Web UI with RemoteMoe**  
* Wait for the line that says "App started successful. Use the app with http://127.0.0.1:7865/ or 127.0.0.1:7865"   
* Click the link that ends with .remote.moe

In [ ]:
#Option 2: Starting the Web UI with RemoteMoe
port = 7865

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python $HOME_FOLDER/Fooocus/entry_with_update.py & ssh -R 80:127.0.0.1:$port -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# Install Models

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/169740?type=Model&format=SafeTensor&size=full&fp=fp16'
model_name = 'ZavyChroma.safetensors'

%cd $checkpoints
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a LoRA to permanent storage
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'

LORA_FOLDER = HOME_FOLDER + '/Fooocus/models/loras'
%cd $LORA_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a model to temporary storage
model_url = 'https://civitai.com/api/download/models/160191?type=Model&format=SafeTensor&size=full&fp=fp16'
model_name = 'YamersRealism.safetensors'

%cd $TEMP_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

if not path.exists(f'{HOME_FOLDER}/Fooocus/models/checkpoints/{model_name}'):
    get_ipython().system(f'ln -s {TEMP_FOLDER}/{model_name} {HOME_FOLDER}/Fooocus/models/checkpoints/')

## Remove corrupted models

In [ ]:
!ls -la $checkpoints
!rm -rf $checkpoints/*.corrupted
!ls -la $checkpoints

## 
## Delete permanent models and LoRAs

In [ ]:
!rm $HOME_FOLDER/Fooocus/models/checkpoints/*
!rm $HOME_FOLDER/Fooocus/models/loras/*